<a href="https://colab.research.google.com/github/gmoriki/C4RA-Python-Tutorials/blob/main/0623/0623_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第7回C4RA勉強会
## DataFrameのデータ結合・マージ

---

## ★ 勉強会の基本方針

* 5月12日(金)から6月30日(金)までの計8回を予定しています。
* 目標は**pandas.DataFrame形式の基本的な操作をマスターすること**です。

* 環境はGoogle Colaboratory(通称Colab)を使用します。Googleアカウントをご用意ください。
  * 各々のローカル環境でも実行可能です。
* 一部、東京大学「Pythonプログラミング入門」の教材を共有しながら勉強会を進めます。
  * 勉強会では**東大教材**と呼びます
  * URL：https://utokyo-ipp.github.io/index.html

* Colab（Jupyter Notebook）の詳しい使い方について、勉強会の中で深入りはしません
  * Colab立ち上げのURL：https://colab.research.google.com/
  * 便利なショートカット：https://blog.kikagaku.co.jp/google-colaboratory-shortcut

* Pythonに関する不明点があれば森木(Pumble,Mail,Twitter等)までご連絡ください
* 勉強会の内容でなくてもOKです（Colabの使い方や環境構築なども）
---

## ■ データの結合

### DataFrameの結合: concat(), merge(), join()

Pythonのpandasライブラリには、DataFrame間で操作を行うための `join()`, `merge()`, `concat()` の3つのメソッドがあります。

それぞれについて詳しく見ていきましょう。

In [ ]:
import pandas as pd

df1 = pd.DataFrame({
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3'],
    'key': ['K0', 'K1', 'K2', 'K3']
})

df2 = pd.DataFrame({
    'C': ['C0', 'C1', 'C2', 'C3'],
    'D': ['D0', 'D1', 'D2', 'D3'],
    'key': ['K0', 'K1', 'K2', 'K3']
})


#### 1. concat()
`concat()` 関数は、DataFrameを特定の軸に沿って結合します。デフォルトでは、DataFrameは縦（軸=0）に結合されます。

共通の列（この場合は 'key'）が重複してしまいます。

In [ ]:
result = pd.concat([df1, df2])
result

軸を1に設定すると、DataFrameは横に結合されます。しかし、共通の列が重複してしまうことに注意が必要です。

In [ ]:
result = pd.concat([df1, df2], axis=1)
result

#### 2. merge()

`merge()` 関数は、一つ以上のキーを基にDataFrameを結合します。SQLのJOIN操作と類似しています。

In [ ]:
# 左結合
result = pd.merge(df1, df2, on='key', how='left')
print(result)

# 右結合
result = pd.merge(df1, df2, on='key', how='right')
print(result)

# 内部結合
result = pd.merge(df1, df2, on='key', how='inner')
print(result)

# 外部結合
result = pd.merge(df1, df2, on='key', how='outer')
print(result)

#### 3. join()

`join()` 関数もDataFrameの結合に使用されますが、キーとしてインデックスを使用します。結合方法は `merge()` 関数と同じく `how` パラメータを使用して制御できます。


In [ ]:
# df1, df2を新たに作成します。今度はインデックスを利用するために 'key' 列は使いません。
df1 = pd.DataFrame({
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3']
}, index=['K0', 'K1', 'K2', 'K3'])

df2 = pd.DataFrame({
    'C': ['C0', 'C1', 'C2', 'C3'],
    'D': ['D0', 'D1', 'D2', 'D3']
}, index=['K0', 'K1', 'K2', 'K3'])

# join操作のデフォルトは左結合です。
result = df1.join(df2)
result

列名が重複している場合、接尾辞を指定して区別することが可能です。

In [ ]:
# df1に新たにC列を追加します。
df1['C'] = ['C1', 'C2', 'C3', 'C4']

# lsuffix, rsuffixパラメータを用いて列名に接尾辞を付けます。
result = df1.join(df2, lsuffix='_df1', rsuffix='_df2')
result

## ■ データのフィルタリング(前処理の仕上げ)

### データのCSV出力

前処理が終了したデータは、将来的な使用のためにCSVファイルとして保存することが一般的です。

以下のコードは、前述のフィルタリングされたデータをCSVファイルとして出力します。


In [ ]:
result

In [ ]:
# データをCSVファイルとして出力します
result.to_csv('filtered_data.csv', index=False)

## ■ オマケ：パイプラインについて

In [ ]:
df_kaken = pd.read_csv('./kaken.nii.ac.jp_2023-06-07_10-57-32.csv')

In [ ]:
# copy
df_kaken_1 = df_kaken.copy()
# こうやって書くと大変

# 欠損値を削除
df_kaken_1 = df_kaken_1.dropna(axis=1, how='any')

# 必要な列だけを選択
df_kaken_1 = df_kaken_1[['研究課題名','研究種目','総配分額']]

# 条件に合う行を選択
mask = (df_kaken_1['総配分額'] > df_kaken_1['総配分額'].mode().values[0]) & (df_kaken_1['研究種目'].str.contains('C'))
df_kaken_1 = df_kaken_1[mask]

# '総配分額'でソート
df_kaken_1 = df_kaken_1.sort_values(by='総配分額')

# インデックスをリセット
df_kaken_1 = df_kaken_1.reset_index(drop=True)
df_kaken_1

In [ ]:
# copy
df_kaken_2 = df_kaken.copy()
# 複数の処理を続けて書くことができます(パイプライン)
df_kaken_2 = df_kaken_2.dropna(axis=1, how='any')[['研究課題名','研究種目','総配分額']][(df_kaken_2['総配分額'] > df_kaken_2['総配分額'].mode().values[0]) & (df_kaken_2['研究種目'].str.contains('C'))].sort_values(by='総配分額').reset_index(drop=True)
df_kaken_2

In [ ]:
# copy
df_kaken_3 = df_kaken.copy()

# 見やすい例1
df_kaken_3 = (df_kaken_3.dropna(axis=1, how='any') \
            [['研究課題名','研究種目','総配分額']] \
            [(df_kaken_3['総配分額'] > df_kaken_3['総配分額'].mode().values[0]) & (df_kaken_3['研究種目'].str.contains('C'))] \
            .sort_values(by='総配分額') \
            .reset_index(drop=True)
            ) 
df_kaken_3

In [ ]:
# copy
df_kaken_4 = df_kaken.copy()

# 見やすい例2
def drop_na_cols(df):
    return df.dropna(axis=1, how='any')

def filter_rows(df):
    return df[(df['総配分額'] > df['総配分額'].mode()[0]) & (df['研究種目'].str.contains('C'))]

def select_columns(df):
    return df[['研究課題名','研究種目','総配分額']]

def sort_values(df):
    return df.sort_values(by='総配分額')

def reset_idx(df):
    return df.reset_index(drop=True)

df_kaken_4 = (df_kaken_4
            .pipe(drop_na_cols)
            .pipe(filter_rows)
            .pipe(select_columns)
            .pipe(sort_values)
            .pipe(reset_idx)
           )
df_kaken_4

In [ ]:
# copy
df_kaken_5 = df_kaken.copy()

# 見やすい例3
def process_dataframe(df):
    df = df.dropna(axis=1, how='any')
    df = df[(df['総配分額'] > df['総配分額'].mode()[0]) & (df['研究種目'].str.contains('C'))]
    df = df[['研究課題名','研究種目','総配分額']]
    df = df.sort_values(by='総配分額')
    df = df.reset_index(drop=True)
    return df

df_kaken_5 = df_kaken_5.pipe(process_dataframe)
df_kaken_5